In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import joblib

#引入自定义函数
from src.untils.forecaster_fuction import check_missing_values

### 江西数据

In [5]:
file_path = '../../data/raw/dataset_1'
data_name_list = os.listdir(file_path)
data_dic = {}
for i in range(len(data_name_list)):
    name_id = data_name_list[i][:-4]
    if name_id in ['cl', 'js', 'jz','lym', 'qss', 'wtl', 'zh']: # 风电场才读取数据
        data_temp = pickle.load(open(file_path + '/' +data_name_list[i], 'rb'))
        data_dic[name_id] = data_temp
joblib.dump(data_dic, '../../data/processed/archive_data/dataset_1/total_data_dic.joblib')
data_dic = joblib.load('../../data/processed/archive_data/dataset_1/total_data_dic.joblib')
check_missing_values(data_dic)
'''数据处理'''
# 提取各场站训练集与测试集
def split_train_test_data_by_time(data_dic, train_ratio=0.8):
    """
    将data_dic中的每个数据框按照时间顺序划分为训练集和测试集。

    参数:
    -----------
    data_dic : dict
        包含各风电场数据框的字典。
    train_ratio : float, 默认=0.8
        用于训练的数据比例（取值0到1之间）。

    返回:
    --------
    train_data_dic : dict
        包含各风电场训练数据框的字典。
    test_data_dic : dict
        包含各风电场测试数据框的字典。
    """
    train_data_dic = {}
    test_data_dic = {}

    for station_id, df in data_dic.items():
        # 确保数据已按时间排序（如果没有，请取消下面的注释）
        # df = df.sort_index()  # 如果索引是时间戳
        # 或者，如果有时间列
        # df = df.sort_values('时间列名')

        # 计算训练集的边界索引
        train_size = int(len(df) * train_ratio)

        # 按照时间顺序划分数据集
        train_df = df.iloc[:train_size].copy()
        test_df = df.iloc[train_size:].copy()

        # 存储划分结果
        train_data_dic[station_id] = train_df
        test_data_dic[station_id] = test_df

        print(f"场站 {station_id}: 训练集大小 = {len(train_df)}, 测试集大小 = {len(test_df)}")

    return train_data_dic, test_data_dic

data_dic = joblib.load('../../data/processed/archive_data/dataset_1/total_data_dic.joblib')
train_data_dic, test_data_dic = split_train_test_data_by_time(data_dic)



缺失值统计信息:
cl: 共有 3549 行缺失值 (占总行数的 10.13%)
  - ws_m: 2013 个缺失值
  - power: 1 个缺失值
  - ws_nwp: 1633 个缺失值
  - tem_nwp: 1633 个缺失值
js: 共有 2013 行缺失值 (占总行数的 5.74%)
  - ws_m: 2013 个缺失值
  - power: 1 个缺失值
  - ws_nwp: 1 个缺失值
  - tem_nwp: 1 个缺失值
jz: 共有 2013 行缺失值 (占总行数的 5.74%)
  - ws_m: 2013 个缺失值
  - power: 1 个缺失值
  - ws_nwp: 385 个缺失值
  - tem_nwp: 385 个缺失值
lym: 共有 4125 行缺失值 (占总行数的 11.77%)
  - ws_m: 2013 个缺失值
  - power: 1 个缺失值
  - ws_nwp: 3841 个缺失值
  - tem_nwp: 3841 个缺失值
qss: 共有 3933 行缺失值 (占总行数的 11.22%)
  - ws_m: 2013 个缺失值
  - power: 1 个缺失值
  - ws_nwp: 2209 个缺失值
  - tem_nwp: 2209 个缺失值
wtl: 共有 3454 行缺失值 (占总行数的 9.86%)
  - ws_m: 2013 个缺失值
  - power: 1 个缺失值
  - ws_nwp: 2018 个缺失值
  - tem_nwp: 2017 个缺失值
zh: 共有 3549 行缺失值 (占总行数的 10.13%)
  - ws_m: 2013 个缺失值
  - power: 1 个缺失值
  - ws_nwp: 1921 个缺失值
  - tem_nwp: 1921 个缺失值

共有 7 个键存在缺失值
1. cl: 3549 行缺失值
2. js: 2013 行缺失值
3. jz: 2013 行缺失值
4. lym: 4125 行缺失值
5. qss: 3933 行缺失值
6. wtl: 3454 行缺失值
7. zh: 3549 行缺失值

第一个含缺失值数据集的部分数据示例:
                     ws_m  power  ws_n

In [3]:

# 保存超参数
job_params = {
    "total_data_path": '../data/processed/archive_data/dataset_1/total_data_dic.joblib',
    "user_id": 'xiaowang',
    "task_id": 1,
    "job_number": 0 # 唯一的作业号，用于定位训练测试数据，跟踪进程
}
with open("../../config/job_config/job_params.json", "w") as f:
    json.dump(job_params, f, indent=4)

